In [1]:
import nltk

from nltk.stem.lancaster import LancasterStemmer

stemmer = LancasterStemmer()

#For tensorflow model
import numpy as np
import tflearn
import tensorflow as tf
import random


Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [2]:
#importing the intents file

import json

with open('intents.json') as json_data:
    intents = json.load(json_data)

In [3]:
words = []
classes = []
documents = []
ignore_words = ['?']

#looping through every intent
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenizing each word in the sentence
        w = nltk.word_tokenize(pattern)
        #add to the wordList
        words.extend(w)
        
        #add to documents
        documents.append((w, intent['tag']))
        
        #add to classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
            
#stem and lower each word and removing duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))


print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)
        

314 documents
47 classes ['Women competitions', 'Women hackathons', 'anxiety', 'bad', 'body image', 'breast cancer', 'bullying', 'can I', 'coping', 'counsellor', 'cure', 'depression', 'feeling', 'funding and financing', 'gender bias and discrimination', 'good', 'goodbye', 'government schemes', 'greeting', 'hygiene', 'imposter syndrome', 'lonely', 'menopause', 'mensuration problems', 'mental illness', 'mood', 'networking and mentorship', 'peer pressure', 'periods', 'pregnancy', 'premenstrual syndrome', 'psychiatrist', 'psychologist', 'relationships', 'scholarships for women', 'self-esteem', 'sexually transmitted infections', 'stress', 'thanks', 'treatment', 'vaginal infection', 'women career opportunities', 'women in military', "women's colleges and scholarships", "women's sports", 'work life balance', 'working women in India']
496 unique stemmed words ['!', "'m", "'s", ',', '.', '5g', ':', 'a', 'about', 'acc', 'adv', 'affect', 'afraid', 'aft', 'ag', 'agit', 'agr', 'aim', 'al', 'allow',

In [4]:
training = []
output = []

#create empty array for output
output_empty = [0] * len(classes)

for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    
    #create bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
        
    
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])
    
random.shuffle(training)
training = np.array(training)

train_x = list(training[:, 0])
train_y = list(training[:, 1])

C:\Users\yashw\AppData\Local\Temp\ipykernel_15696\987945108.py:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [5]:
# train_x example: [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1] 
# train_y example: [0, 0, 1, 0, 0, 0, 0, 0, 0]

In [6]:
tf.compat.v1.reset_default_graph()
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
# Start training (apply gradient descent algorithm)
model.fit(train_x, train_y, n_epoch=2000, batch_size=8, show_metric=True)
model.save('model.tflearn')

Training Step: 79999  | total loss: 0.02809 | time: 0.145s
| Adam | epoch: 2000 | loss: 0.02809 - acc: 0.9752 -- iter: 312/314
Training Step: 80000  | total loss: 0.02529 | time: 0.150s
| Adam | epoch: 2000 | loss: 0.02529 - acc: 0.9777 -- iter: 314/314
--
INFO:tensorflow:D:\study\sem5\nlp\contextBasedChatbot\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [7]:

# save all of our data structures
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )

In [8]:
# restore all of our data structures
import pickle
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

# import our chat-bot intents file
import json
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [9]:
# load our saved model
model.load('./model.tflearn')

INFO:tensorflow:Restoring parameters from D:\study\sem5\nlp\contextBasedChatbot\model.tflearn


In [10]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [11]:
# create a data structure to hold user context
context = {}

ERROR_THRESHOLD = 0.25
def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    print("results",results)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    print("return_list", return_list)
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # set context for this intent if necessary
                    if 'context_set' in i:
                        if show_details: print ('context:', i['context_set'])
                        context[userID] = i['context_set']

                    # check if this intent is contextual and applies to this user's conversation
                    if not 'context_filter' in i or \
                        (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        if show_details: print ('tag:', i['tag'])
                        # a random response from the intent
                        return print(random.choice(i['responses']))
                    else:
                        print("I do not understand")
    
            results.pop(0)
    else:
        print("I do not understand")

In [12]:
response('My parents dont support me')

results [[6, 0.81899565]]
return_list [('bullying', 0.81899565)]
There are resources available to help with bullying, including hotlines and online support groups. It's important to take action and get the help you need to feel safe and supported.


In [13]:
response("i feel depressed")

results [[25, 0.51082295], [3, 0.479516]]
return_list [('mood', 0.51082295), ('bad', 0.479516)]
I'm sorry to hear that. Can you tell me more about what's going on?


In [14]:
response("My friends don't like me")

results [[35, 0.99908864]]
return_list [('self-esteem', 0.99908864)]
It can be helpful to challenge negative self-talk by writing down your accomplishments and positive traits. You can also try talking to a trusted friend or counselor about how you're feeling.


In [19]:
response("i am going to have a baby")

results [[25, 0.9924614]]
return_list [('mood', 0.9924614)]
It can be tough to feel sad. Would you like to talk about what's been bothering you?
